#Importing modules

In [ ]:
#IMPORTING LIBRARIES
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import joblib

#IMPORTING DATAPREPROCESSING TOOLS
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

#IMPORTING MODELS
from sklearn.ensemble import AdaBoostRegressor
from sklearn.linear_model import ElasticNet
from sklearn.ensemble import GradientBoostingRegressor 
from sklearn.neighbors import KNeighborsRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.linear_model import Lasso
from sklearn.cross_decomposition import PLSRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge

#IMPORTING CROSS VALIDATION TOOLS
from sklearn.model_selection import KFold	
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import RandomizedSearchCV

#FEATURE TOOLS
from sklearn.feature_selection import RFE
from sklearn.inspection import permutation_importance

In [ ]:
#NAMING MODELS
abr = AdaBoostRegressor()
en = ElasticNet()
gbr = GradientBoostingRegressor()
knr = KNeighborsRegressor()
kr = KernelRidge()
las = Lasso()
pls = PLSRegression()
rfr = RandomForestRegressor()
rdg = Ridge()

allmodels=[abr, en, gbr, knr, kr, las, pls, rfr, rdg]

In [ ]:
#LOADING DATA
df1 = pd.read_csv('dataset.csv')
df2 = pd.read_csv('deltaG_H.csv')
df = pd.merge(df1, df2, on = 'Mxenes')
#dropping unnecessary columns
df = df.drop(['G_X 17'], axis=1)

#Feature Subsets generation

In [ ]:
#DIVIDING DATASET INTO FEATURE SETS
set1_trainx = df.iloc[:,0:62]                                       #SET-1 ATOMISTIC FEATURES 
set2_trainx = df.iloc[:,62:83]                                      #SET-2 SURFACE (STRUCTURAL + ELECTRONIC) FEATURES
set3_trainx = df.iloc[:,83:-1]                                      #SET-3 STATISTICAL FEATURES 
set4_trainx = pd.concat([set1_trainx, set2_trainx], axis = 1)       #SET-4 = 1+2
set5_trainx = pd.concat([set1_trainx, set3_trainx], axis = 1)       #SET-5 = 1+3
set6_trainx = pd.concat([set2_trainx, set3_trainx], axis = 1)       #SET-6 = 2+3
set7_trainx = df.iloc[:,:-1]                                        #SET-7 = 1+2+3 

#Cross Validation for all feature sets

In [ ]:
#SET 1-7 Cross validation
#rename set1 each time for each set CV data
set_r2 = []
set_mae = []

for model in allmodels:
  train_x = set1_trainx 
  train_y = df.iloc[:,-1]
  sf = MinMaxScaler()
  enc = OneHotEncoder()
  numerical = train_x.select_dtypes(exclude='bool').columns
  categorical = train_x.select_dtypes(include='object').columns
  t = [("cat", enc, categorical), ("num", sf, numerical)]
  ct = ColumnTransformer(transformers=t)
  train_x_norm = ct.fit_transform(train_x)
  model.fit(train_x_norm, train_y)
  
  print(model)
  cv = KFold(n_splits=10, shuffle=True, random_state=None)
  score = cross_val_score(model, train_x[selected_features], train_y, scoring='neg_mean_absolute_error',cv=cv, n_jobs=None)
  score2 = cross_val_score(model, train_x[selected_features], train_y, scoring='r2',cv=cv, n_jobs=None)
  print('avg MAE:', sum(score)/len(score), 'avg R2:', sum(score2)/len(score2))

  set_mae.append(round(score,3))
  set_r2.append(round(score2,3))

  model.fit(train_x, train_y)
  predictions = model.predict(train_x).flatten()
  
  a = plt.axes(aspect='equal')
  plt.scatter(train_y, predictions)
  plt.xlabel('True Values [gibbs free energy]')
  plt.ylabel('Predictions [gibbs free energy]')
  lims = [-5,5]
  plt.xlim(lims)
  plt.ylim(lims)
  
  fig = plt.plot(lims, lims)
  plt.show()
  print('------------------------------------------------------------------------------------------------------------------')

set1cv = pd.DataFrame()
set1cv['model'] = allmodels
set1cv['r2'] = set_r2
set1cv['mae'] = set_mae
set1cv.to_csv('set1.csv', index = False)
del set_r2
del set_mae

#Test-Train Verification on Set-7

In [ ]:
dataset = df
X = dataset.iloc[:, :-1].values
y = dataset.iloc[:, -1].values

train_x, test_x, train_y, test_y = train_test_split(X, y, test_size = 0.2, random_state = 0)
for model in allmodels:
  r2list = []
  maelist = []
  r2testlist = []
  maetestlist = [] 
  print(model)
  
  model.fit(train_x, train_y)
  train_y_pred = model.predict(train_x)
  r2 = r2_score(train_y, train_y_pred)
  maer = abs(mae(train_y, train_y_pred))
  r2list.append(r2)
  maelist.append(maer)
  test_y_pred = model.predict(test_x)
  r2test = r2_score(test_y, test_y_pred)
  maertest = abs(mae(test_y, test_y_pred))
  r2testlist.append(r2test)
  maetestlist.append(maertest)
  print('------------------------------------------------------------------------------------------------------------------')

set7tt = pd.DataFrame()
set7tt['model'] = str(model) 
set7tt['r2'] = r2list
set7tt['mae'] = maelist
set7tt['test r2'] =  r2testlist
set7tt['test mae'] = maetestlist
set7tt.to_csv('set7 test train data.csv')
del r2list
del maelist
del r2testlist
del maetestlist

#Recursive Feature Elimination

In [ ]:
#RFE ON BEST MODELS FOR FEATURE SELECTION
train_x = df.iloc[:,:-1]
train_y = df.iloc[:,-1]
numerical = train_x.select_dtypes(exclude='bool').columns
categorical = train_x.select_dtypes(include='object').columns

t = [("cat", enc, categorical), ("num", sf, numerical)]
ct = ColumnTransformer(transformers=t)
train_x_norm = ct.fit_transform(train_x)

In [ ]:
#RFE on RFR Model
#running a 'for' loop to check best metrics scores by varying the number of features
numf = [60, 55, 50, 45, 40, 35, 30, 25, 20, 15, 10]
model = RandomForestRegressor()

for i in numf:
  selector = RFE(model, n_features_to_select=i, step=10, verbose=0, importance_getter='auto')
  selector.fit(train_x_norm, train_y)
  selector_support = selector.get_support()
  selected_features = train_x.loc[:, selector_support].columns.tolist()
  print(selected_features)
  cv = KFold(n_splits=10, shuffle=True, random_state=None)
  score = cross_val_score(model, train_x[selected_features], train_y, scoring='neg_mean_absolute_error',cv=cv, n_jobs=None)
  score2 = cross_val_score(model, train_x[selected_features], train_y, scoring='r2',cv=cv, n_jobs=None)
  mae = (sum(score)/len(score))
  r2 = (sum(score2)/len(score2))
  print('num of features: ', i)
  print('MAE:', round(mae,3))
  print('R2:', round(r2,3))
  print('--------------------------------------------------------------------------------------------------------------------------')


In [ ]:
#best metrics score when number of features were in between 20-30. Thus, checking in that range

numf = [30, 29, 28, 27, 26, 25, 24, 23, 22, 21, 20]
for i in numf:
  selector = RFE(model, n_features_to_select=i, step=10, verbose=0, importance_getter='auto')
  selector.fit(train_x_norm, train_y)
  selector_support = selector.get_support()
  selected_features = train_x.loc[:, selector_support].columns.tolist()
  print(selected_features)
  cv = KFold(n_splits=10, shuffle=True, random_state=None)
  score = cross_val_score(model, train_x[selected_features], train_y, scoring='neg_mean_absolute_error',cv=cv, n_jobs=None)
  score2 = cross_val_score(model, train_x[selected_features], train_y, scoring='r2',cv=cv, n_jobs=None)
  mae = (sum(score)/len(score))
  r2 = (sum(score2)/len(score2))
  print('num of features: ', i)
  print('MAE:', round(mae,3))
  print('R2:', round(r2,3))
  print('--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------')

In [ ]:
#RFE on GBR model
#running a 'for' loop to check best metrics scores by varying the number of features
model = GradientBoostingRegressor()
numf = [60, 55, 50, 45, 40, 35, 30, 25, 20, 15, 10]

for i in numf:
  selector = RFE(model, n_features_to_select=i, step=10, verbose=0, importance_getter='auto')
  selector.fit(train_x_norm, train_y)
  selector_support = selector.get_support()
  selected_features = train_x.loc[:, selector_support].columns.tolist()
  print(selected_features)
  cv = KFold(n_splits=10, shuffle=True, random_state=None)
  score = cross_val_score(model, train_x[selected_features], train_y, scoring='neg_mean_absolute_error',cv=cv, n_jobs=None)
  score2 = cross_val_score(model, train_x[selected_features], train_y, scoring='r2',cv=cv, n_jobs=None)
  mae = (sum(score)/len(score))
  r2 = (sum(score2)/len(score2))
  print('num of features: ', i)
  print('MAE:', round(mae,3))
  print('R2:', round(r2,3))
  print('--------------------------------------------------------------------------------------------------------------------------')


In [ ]:
#best metrics score when number of features were in between 20-30. Thus, checking in that range

numf = [35, 34, 33, 32, 31, 30, 29, 28, 27, 26, 25]
for i in numf:
  selector = RFE(model, n_features_to_select=i, step=10, verbose=0, importance_getter='auto')
  selector.fit(train_x_norm, train_y)
  selector_support = selector.get_support()
  selected_features = train_x.loc[:, selector_support].columns.tolist()
  print(selected_features)
  cv = KFold(n_splits=10, shuffle=True, random_state=None)
  score = cross_val_score(model, train_x[selected_features], train_y, scoring='neg_mean_absolute_error',cv=cv, n_jobs=None)
  score2 = cross_val_score(model, train_x[selected_features], train_y, scoring='r2',cv=cv, n_jobs=None)
  mae = (sum(score)/len(score))
  r2 = (sum(score2)/len(score2))
  print('num of features: ', i)
  print('MAE:', round(mae,3))
  print('R2:', round(r2,3))
  print('--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------')

#Hyperparameter optimization

In [ ]:
#Hyperparameter optimization (tuning) for RFR Model

n_estimators = [20, 40, 60, 80, 100, 200, 300, 400, 600, 700, 800, 1000]
max_features = [None, 'sqrt', 'log2', 25, 15, 10, 5, 2]
max_depth = [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 200, 300, 400, 500, 1000, 5000, 10000]
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 4]
bootstrap = [True, False]

random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(random_grid)

model = RandomForestRegressor()

#proplist = ['atomic no. m', 'valance electron t', 'electron affinity (eV) t', 'MP (in degree C) t', 'valance electron avg', 'BP (in degree C) t', 'layer thickness', 'bondlength m-t', 'bondlength m-x', 'dbandcenter', 'x-x sq', 'm-m2 sq', 't-t2 sq', 't-t', 'm-m', 'x-x', 'm-m2', 'wfn', 'max bl', 'dbandcenterpr', 'dbandcenter var', 'ea var', 'dbandcenter std', 'distance x-x var', 'gibbs free energy']
proplist = ['N_M', 'VE_T', 'EA_T', 'MP_T', 'VE avg', 'BP_T', 'LT', 'l_M-T', 'l_M-X', 'dbc', 'd_X-X sq', 'd_M-M2 sq', 'd_T1-T2 sq', 'd_T-T', 'd_M-M', 'd_X-X', 'd_M-M2', 'WF', 'l_max', 'dbc avg', 'dbc var', 'EA_T var', 'dbc std', 'd_X-X var', 'deltaG_H']

#proplist =  selected_features + target_property
df = df[proplist]
train_x = df.iloc[:,:-1]
train_y = df.iloc[:,-1]
sf = MinMaxScaler()
enc = OneHotEncoder()
numerical = train_x.select_dtypes(exclude='bool').columns
categorical = train_x.select_dtypes(include='object').columns
t = [("cat", enc, categorical), ("num", sf, numerical)]
ct = ColumnTransformer(transformers=t)
train_x_norm = ct.fit_transform(train_x)

rf_random = RandomizedSearchCV(estimator = model, param_distributions = random_grid, n_iter = 100, cv = 10, verbose=2, random_state=0, scoring='r2', return_train_score=True)
# Fit the random search model
rf_random.fit(train_x_norm, train_y)
print(rf_random.best_params_)

In [ ]:
#hyperparameter optimization (tuning) for GBR model

learning_rate=[0.001, 0.005, 0.01, 0.015, 0.1, 0.5, 1]
n_estimators = [20, 40, 60, 80, 100, 200, 300, 400, 600, 700, 800, 1000]
max_features = [None, 'sqrt', 'log2', 25, 15, 10, 5, 2]
max_depth = [3, 5, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 200, 300, 400, 500, 1000, 5000, 10000]
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 4]
bootstrap = [True, False]

random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(random_grid)

model = GradientBoostingRegressor()

#selected_features_gbr = ['metallic radii (pm) m', 'valance electron t', 'first ionization potential (eV) t', 'electron affinity (eV) t', 'MP (in degree C) t', 'BP (in degree C) t', 'electronegativity t', 'cohesive_en', 'layer thickness', 'bondlength m-t', 'bondlength m-x', 'dbandcenter', 'distance x-x', 'distance m-m2', 'nearest neighbor', 'work_fn', 'min bondlength', 'max bondlength', 'electron affinity var', 'cohesive_en std', 'distance m2-t avg', 'dbandcenter avg', 'dbandcenter var', 'dbandcenter std', 'distance t-t2 std', 'wfn std', 'valance electron t avg', 'normalized surface wt', 't-t sq', 'm-m2 sq', 'gibbs free energy']
proplist = ['r_M', 'VE_T', 'IE_T', 'EA_T', 'MP_T', 'BP_T', 'EA_T', 'W', 'E_coh', 'LT', 'l_M-T', 'l_M-X', 'dbc ', 'd_X-X', 'd_M-M2', 'd_NN', 'WF', 'l_min', 'l_max', 'EA var', 'E_coh std', 'l_M2-T avg', 'dbc avg', 'dbc var', 'dbc std', 'd_T1-T2 std', 'WF std', 'VE_T avg', 'd_T-T sq', 'd_M-M2 sq', 'deltaG_H']
#proplist =  selected_features + target_property
df = df[proplist]
train_x = df.iloc[:,:-1]
train_y = df.iloc[:,-1]
sf = MinMaxScaler()
enc = OneHotEncoder()
numerical = train_x.select_dtypes(exclude='bool').columns
categorical = train_x.select_dtypes(include='object').columns
t = [("cat", enc, categorical), ("num", sf, numerical)]
ct = ColumnTransformer(transformers=t)
train_x_norm = ct.fit_transform(train_x)

# search across 100 different combinations, and use all available cores
gbr_random = RandomizedSearchCV(estimator = model, param_distributions = random_grid, n_iter = 100, cv = 10, verbose=2, random_state=0, scoring='r2', return_train_score=True)
# Fit the random search model
gbr_random.fit(train_x_norm, train_y)
print(gbr_random.best_params_)

#Target property prediction

In [ ]:
#FINAL RFR MODEL
model = RandomForestRegressor(n_estimators=1000, min_samples_split=5, min_samples_leaf=2, max_features=15, max_depth=500, bootstrap= True)
model.fit(train_x_norm, train_y)

cv = KFold(n_splits=10, shuffle=True, random_state=0)
score = cross_val_score(model, train_x, train_y, scoring='neg_mean_absolute_error',cv=cv, n_jobs=None)
score2 = cross_val_score(model, train_x, train_y, scoring='r2',cv=cv, n_jobs=None)
s1 = sum(score)/len(score)
s2 = sum(score2)/len(score2)
print(round(s1,3), round(s2,3))

In [ ]:
#SAVING MODEL (for further use)
joblib.dump(model, 'rfr_model.joblib')

#PREDICTING GIBBS FREE ENERGY FOR 4500 CATALYSTS 
selected_features_rfr = ['N_M', 'V_T', 'EA_T', 'MP_T', 'V avg', 'BP_T', 'LT', 'l_M-T', 'l_M-X', 'dbc', 'd_X-X sq', 'd_M-M2 sq', 'd_T1-T2 sq', 'd_T-T', 'd_M-M', 'd_X-X', 'd_M-M2', 'WF', 'L_max', 'dbc avg', 'dbc var', 'EA_T var', 'dbc std', 'd_X-X var']
x_all = df1[selected_features_rfr] #feature dataset
model.fit(train_x, train_y)
predictions_rfr = model.predict(x_all) #predicted target property 

In [ ]:
#FINAL GBR MODEL
model = GradientBoostingRegressor(n_estimators= 400, min_samples_split= 2, min_samples_leaf= 10, max_features= 'sqrt', max_depth= 1000, learning_rate= 0.015)
model.fit(train_x_norm, train_y)

cv = KFold(n_splits=10, shuffle=True, random_state=0)
score = cross_val_score(model, train_x, train_y, scoring='neg_mean_absolute_error',cv=cv, n_jobs=None)
score2 = cross_val_score(model, train_x, train_y, scoring='r2',cv=cv, n_jobs=None)
s1 = sum(score)/len(score)
s2 = sum(score2)/len(score2)
print(s1, s2)
print(round(s1,3), round(s2,3))

In [ ]:
#SAVING MODEL (for further use)
joblib.dump(model, 'gbr_model.joblib')

#PREDICTING GIBBS FREE ENERGY FOR 4500 CATALYSTS
selected_features_gbr = ['r_M', 'V_T', 'IE_T', 'EA_T', 'MP_T', 'BP_T', 'EA_T', 'W', 'E_coh', 'LT', 'l_M-T', 'l_M-X', 'dbc ', 'd_X-X', 'd_M-M2', 'd_NN', 'WF', 'l_min', 'l_max', 'EA var', 'E_coh std', 'l_M2-T avg', 'dbc avg', 'dbc var', 'dbc std', 'd_T1-T2 std', 'WF std', 'V_T avg', 'd_T-T sq', 'd_M-M2 sq']

x_all = df1[selected_features_gbr] #feature dataset
model.fit(train_x, train_y)
predictions_gbr = model.predict(x_all) #predicted target property

#Finding Key Descriptors

In [ ]:
start_time = time.time()
result = permutation_importance(
    model, train_x, train_y, n_repeats=10, random_state=42, n_jobs=2
)
elapsed_time = time.time() - start_time
print(f"Elapsed time to compute the importances: {elapsed_time:.3f} seconds")

feature_importances = pd.Series(result.importances_mean, index=train_x.columns)
print(feature_importances)